<a href="https://colab.research.google.com/github/Shriansh16/NLP/blob/main/MESSAGE_SPAM_CLASSIFICATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('/content/SMSSpamCollection.txt',sep='\t',names=['label','message'])

In [3]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.shape

(5572, 2)

In [5]:
df['label'].unique()

array(['ham', 'spam'], dtype=object)

In [6]:
df.isnull().sum()

label      0
message    0
dtype: int64

In [7]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [8]:
df.duplicated().sum()

403

In [9]:
df=df.drop_duplicates()

In [10]:
df.reset_index(drop=True,inplace=True)

In [11]:
df['label'].value_counts()

ham     4516
spam     653
Name: label, dtype: int64

In [12]:
df['message'][500]

"You should know now. So how's anthony. Are you bringing money. I've school fees to pay and rent and stuff like that. Thats why i need your help. A friend in need....|"

##TEXT PREPREOCESSING

In [13]:
import re
import nltk


In [14]:
stop_words=nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
from nltk.corpus import stopwords

In [16]:
from nltk.stem import PorterStemmer

In [17]:
ps=PorterStemmer()

In [21]:
corpus=[]
for i in range(len(df)):
  temp=re.sub('^a-zA-Z0-9',' ',df['message'][i])
  temp=temp.lower()
  temp=temp.split()
  temp=[ps.stem(k) for k in temp if k not in stopwords.words('english')]
  temp=' '.join(temp)
  corpus.append(temp)


###using bag of words

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [22]:
cv=CountVectorizer(max_features=2500,binary=True) #only the top 2500 most frequent words will be considered.

In [23]:
X=cv.fit_transform(corpus).toarray()

In [24]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [40]:
y=df['label'].apply(lambda x: 0 if x=='ham' else 1)

In [41]:
y

0       0
1       0
2       1
3       0
4       0
       ..
5164    1
5165    0
5166    0
5167    0
5168    0
Name: label, Length: 5169, dtype: int64

In [42]:
type(y)

pandas.core.series.Series

In [43]:
y=y.values

In [44]:
y

array([0, 0, 1, ..., 0, 0, 0])

In [45]:
from sklearn.model_selection import train_test_split

In [46]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=43)

In [48]:
from sklearn.ensemble import RandomForestClassifier

In [49]:
rf=RandomForestClassifier()

In [50]:
rf.fit(X_train,y_train)

RandomForestClassifier()

In [51]:
y_pred=rf.predict(X_test)


In [54]:
from sklearn.metrics import accuracy_score,classification_report

In [53]:
accuracy_score(y_test,y_pred)

0.9758220502901354

In [55]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       890
           1       0.98      0.85      0.91       144

    accuracy                           0.98      1034
   macro avg       0.98      0.92      0.95      1034
weighted avg       0.98      0.98      0.98      1034



In [57]:
# creating tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer

In [58]:
tfidf=TfidfVectorizer(max_features=2500)

In [61]:
X=tfidf.fit_transform(corpus).toarray()

In [62]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [64]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [67]:
model2=RandomForestClassifier()

In [68]:
model2.fit(X_train,y_train)

RandomForestClassifier()

In [69]:
y_pred=model2.predict(X_test)

In [70]:
accuracy_score(y_test,y_pred)

0.9816247582205029

##WORD2VEC

In [71]:
!pip install gensim

In [72]:
import gensim

In [73]:
from gensim.models import Word2Vec,KeyedVectors

In [74]:
import gensim.downloader as api

In [75]:
wv=api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [80]:
embedding=[]
for i in corpus:
  words=nltk.word_tokenize(i)
  embedded_words=[wv[word] for word in words if word in wv.vocab]
  sent=' '.join(embedded_words)
  embedding.append(sent)



AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [79]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
for token in tokens:
        if token in pretrained_embeddings:
            embeddings.append(pretrained_embeddings[token])